In [1]:
import mariadb
import os
from columnar import columnar
from dotenv import load_dotenv

_ = load_dotenv("config.env")

In [2]:
connection = mariadb.connect(
    host="127.0.0.1",
    port=23306,
    user=os.getenv("user"),
    password=os.getenv("password"),
    autocommit=False,
)
cursor = connection.cursor()

In [3]:
def total_sales_per_sku(db_name: str, table_name: str):
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS {db_name}.total_sales_per_sku (sku INT, count INT)
        SELECT sku, COUNT(*) AS "count"
                FROM {db_name}.{table_name}
                GROUP BY sku
                ORDER BY COUNT(*) DESC
        """.format(
            db_name=db_name,
            table_name=table_name,
        )
    )

In [4]:
total_sales_per_sku("team8", "transactions")
total_sales_per_sku("team7", "purchases")

In [16]:
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS data_mart.total_sales_per_sku (sku INT, count INT, rank INT, team7_count INT, team7_rank INT, team8_count INT, team8_rank INT)
    SELECT sku, total as count, rank, team7_count, team7_rank, team8_count, team8_rank FROM
        (SELECT * FROM
            (SELECT sku, CAST(SUM(count) AS INTEGER) AS total, ROW_NUMBER() OVER (ORDER BY total DESC) AS rank
                FROM (SELECT sku, count FROM team8.total_sales_per_sku
                      UNION ALL
                      SELECT sku, count FROM team7.total_sales_per_sku) total
                GROUP BY total.sku) global
            JOIN (SELECT sku as team8_sku, count AS team8_count, ROW_NUMBER() OVER (ORDER BY count DESC) AS team8_rank FROM team8.total_sales_per_sku) team8
            ON sku = team8_sku) gl_team8
        JOIN (SELECT sku AS team7_sku, count AS team7_count, ROW_NUMBER() OVER (ORDER BY count DESC) AS team7_rank FROM team7.total_sales_per_sku) team7
        ON sku = team7_sku
        ORDER BY rank
    """
)

In [30]:
cursor.execute(
    """
    SELECT product_catalog.sku, CONCAT(manufacturer, ": ", product_name, " (", size, ")") AS product, count, rank, team7_count, team7_rank, team8_count, team8_rank
    FROM data_mart.total_sales_per_sku
    JOIN data_mart.product_catalog ON product_catalog.sku = total_sales_per_sku.sku
    ORDER BY rank
    LIMIT 25
    """
)
tuples = cursor.fetchall()
lists = [list(x) for x in tuples]
table = columnar(lists, ["sku", "product", "count", "rank", "team7_count", "team7_rank", "team8_count", "team8_rank"], no_borders=True, terminal_width=150)
print(table)

                  
  SKU       PRODUCT                                               COUNT  RANK   TEAM7_COUNT  TEAM7_RANK  TEAM8_COUNT  TEAM8_RANK  
    
  42360001  Rowan Dairy: Whole Milk Milk (1/2 gal)                97792  1      49027        1           48765        1           
  42357001  Rowan Dairy: 2.00% Milk (1 gal)                       97685  2      48946        2           48739        3           
  42356001  Rowan Dairy: 1.00% Milk (1/2 gal)                     97348  3      48589        5           48759        2           
  42355001  Rowan Dairy: 1.00% Milk (1 gal)                       97198  4      48788        3           48410        5           
  42358001  Rowan Dairy: 2.00% Milk (1/2 gal)                     97024  5      48374        6           48650        4           
  42359001  Rowan Dairy: Whole Milk Milk (1 gal)                  97006  6      48629        4           48377        6           
  42313001  Smuckers: Jelly Grape (18 oz)                  